# Benchmarking Different Methods 2

In this task we will benchmark a number of different methods evaluating both performance and quality 
of solutions. 

Here we continue exploration of methods.

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import strawberryfields as sf
from strawberryfields.ops import *
import matplotlib

font = {'family' : 'Dejavu Sans',
        'weight' : 'bold',
        'size'   : 22}
matplotlib.rc('font', **font)


## Simple QAOA with STRAWBERRY FIELDS

In [5]:
import flatnetwork_simple as fns

In [7]:
#Setup
K = 20
dt = 1.0

#Parameters
mu = lambda t,i: 1.0
rho = 1.0

#Annealing params
J = 1.0
theta = -J*dt/K

In [8]:
L = [1,3,1]
d = 3
fnet = fns.FlatNetwork(L,d)
fnet.loadParams(mu, rho)
fnet.make_mpos()

S, D = fnet.getHamiltonian()

Max MPO dim:  6


In [9]:
print(S)
print(D)
print(np.prod(L))

{(0, 0): -1.0, (1, 1): -1.0, (2, 2): -1.0}
{(0, 0): 0.1111111111111111, (0, 1): 0.2222222222222222, (0, 2): 0.2222222222222222, (1, 1): 0.1111111111111111, (1, 2): 0.2222222222222222, (2, 2): 0.1111111111111111}
3


In [10]:
nmodes = int(np.prod(L))
ham_simulation = sf.Program(nmodes)

with ham_simulation.context as q:
    #Prepare the initial state
    Fock(3) | q[0] #Max particles allowed in system also upper bound on particle coun
    
    for i in range(K):
        BSgate(theta, np.pi/2) | (q[0], q[1])
                        
        for pair in S:
            Rgate(-1j*S[pair]/K*dt) | q[pair[0]]
        for pair in D:
            if(pair[0] == pair[1]):
                Kgate(-1j*D[pair]/K*dt) | q[pair[0]]
            else:
                CKgate(-1j*D[pair]/K*dt) | (q[pair[0]], q[pair[1]])
        

In [13]:
eng = sf.Engine(backend="fock", backend_options={"cutoff_dim": 5})

In [14]:
results = eng.run(ham_simulation)

In [15]:
state = results.state
print("P(|0, 0>) = ", state.fock_prob([0, 1, 2]))
print("P(|0, 1>) = ", state.fock_prob([0, 2, 1]))
print("P(|0, 1>) = ", state.fock_prob([1, 2, 0]))
print("P(|0, 1>) = ", state.fock_prob([1, 0, 2]))
print("P(|0, 2>) = ", state.fock_prob([1, 2, 0]))
print("P(|0, 2>) = ", state.fock_prob([2, 0, 1]))
print("P(|0, 2>) = ", state.fock_prob([2, 1, 0]))

P(|0, 0>) =  0.0
P(|0, 1>) =  0.0
P(|0, 1>) =  0.008042175585530744
P(|0, 1>) =  0.0
P(|0, 2>) =  0.008042175585530744
P(|0, 2>) =  0.0
P(|0, 2>) =  0.0033156516933681487


In [16]:
state.all_fock_probs()

array([[[0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.00650215, 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ]],

       [[0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.00804218, 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ]],

       [[0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.00331565, 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.     